In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import collections
import random

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.18.0


In [24]:
text = """
The quick brown fox jumps over the lazy dog.
A journey of a thousand miles begins with a single step.
Generative AI is transforming how we interact with technology.
Neural networks are powerful tools for pattern recognition.
Deep learning models require vast amounts of data.
"""

# Convert text to lowercase for consistency
text = text.lower()

# Create a sorted list of unique characters in the text
vocab = sorted(list(set(text)))
char_to_int = {char: i for i, char in enumerate(vocab)}
int_to_char = {i: char for i, char in enumerate(vocab)}

vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")
print(f"Vocabulary: {vocab}")

# Prepare training sequences
seq_length = 50 # Length of input sequences
data_X = [] # Input sequences
data_y = [] # Output characters (next character in sequence)

# Create sequences of characters and their corresponding next character
for i in range(0, len(text) - seq_length):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    data_X.append([char_to_int[char] for char in seq_in])
    data_y.append(char_to_int[seq_out])

n_patterns = len(data_X)
print(f"Total patterns: {n_patterns}")

# Reshape X to be [samples, time steps] for Embedding layer input
X = np.reshape(data_X, (n_patterns, seq_length))

# One-hot encode the output variable (y) - not needed for sparse_categorical_crossentropy
y = np.array(data_y)

Vocabulary size: 29
Vocabulary: ['\n', ' ', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total patterns: 227


In [25]:
model = Sequential([
    # Embedding layer maps integer indices to dense vectors
    Embedding(input_dim=vocab_size, output_dim=256, input_length=seq_length, input_shape=(seq_length,)),
    # LSTM layer to learn sequential dependencies
    LSTM(256, return_sequences=True), # return_sequences=True for stacking LSTMs
    Dropout(0.2), # Dropout for regularization
    LSTM(256), # Last LSTM layer does not return sequences
    Dropout(0.2),
    # Dense layer for output, with softmax activation for probability distribution over vocabulary
    Dense(vocab_size, activation='softmax')
])

# Use Adam optimizer
optimizer = Adam(learning_rate=0.005)
# Use sparse_categorical_crossentropy because y is integer-encoded, not one-hot
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 50, 256)        │         7,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 50, 256)        │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 50, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 29)             │         7,453 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,065,501 (4.06 MB)

 Trainable params: 1,065,501 (4.06 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
print("\nStarting model training...")
# Train the model
# Using a small number of epochs for demonstration.
# For better results, increase epochs and use callbacks like ModelCheckpoint.
history = model.fit(X, y, epochs=20, batch_size=128, verbose=1)
print("Model training complete.")


Starting model training...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 771ms/step - loss: 3.3485
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 779ms/step - loss: 3.5544
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 740ms/step - loss: 3.0617
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 799ms/step - loss: 2.9766
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 2.9441
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 789ms/step - loss: 2.9088
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 780ms/step - loss: 2.9585
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 803ms/step - loss: 2.9534
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 2.9419
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 793ms/step - loss: 3.1306
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 773ms/step - loss: 2.8598
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 800ms/step - loss: 2.7912
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step - loss: 2.7237
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 2.6956
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 797ms/step - loss: 2.6161


In [27]:
def generate_text(model, seed_text, num_chars_to_generate=200, temperature=1.0):
    """
    Generates text using the trained LSTM model.

    Args:
        model (tf.keras.Model): The trained Keras model.
        seed_text (str): The initial text to start generation from.
        num_chars_to_generate (int): The number of characters to generate.
        temperature (float): Controls the randomness of predictions.
                             Lower temp (e.g., 0.2) makes text more predictable.
                             Higher temp (e.g., 1.0) makes text more random/creative.
    Returns:
        str: The generated text.
    """
    generated_text = seed_text.lower()
    pattern = [char_to_int[char] for char in generated_text[-seq_length:]] # Use last part of seed if too long

    print(f"\nGenerating text with seed: \"{seed_text}\"")
    print(f"Temperature: {temperature}")

    for i in range(num_chars_to_generate):
        # Reshape input for the model: (1, seq_length, 1)
        x = np.reshape(pattern, (1, len(pattern), 1))
        # Predict probabilities for the next character
        prediction_probabilities = model.predict(x, verbose=0)[0]

        # Apply temperature to probabilities
        prediction_probabilities = np.log(prediction_probabilities) / temperature
        exp_preds = np.exp(prediction_probabilities)
        prediction_probabilities = exp_preds / np.sum(exp_preds)

        # Sample the next character based on probabilities
        next_char_int = np.random.choice(len(vocab), p=prediction_probabilities)
        next_char = int_to_char[next_char_int]

        generated_text += next_char
        # Update the pattern for the next prediction
        pattern.append(next_char_int)
        pattern = pattern[1:len(pattern)] # Keep the pattern length consistent

    return generated_text

In [28]:
print("\n" + "="*50)
print("Demonstrating Text Generation")
print("="*50 + "\n")

# Example 1: Generate text with a common seed
seed1 = "the quick brown fox"
generated_output1 = generate_text(model, seed1, num_chars_to_generate=150, temperature=0.5)
print(f"\nGenerated Text 1:\n{generated_output1}")
print("\n" + "="*50 + "\n")

# Example 2: Generate text with a different seed and higher temperature
seed2 = "neural networks"
generated_output2 = generate_text(model, seed2, num_chars_to_generate=150, temperature=1.0)
print(f"\nGenerated Text 2:\n{generated_output2}")
print("\n" + "="*50 + "\n")

# Example 3: Generate text with a very short seed
seed3 = "deep learning"
generated_output3 = generate_text(model, seed3, num_chars_to_generate=150, temperature=0.7)
print(f"\nGenerated Text 3:\n{generated_output3}")
print("\n" + "="*50 + "\n")


Demonstrating Text Generation


Generating text with seed: "the quick brown fox"
Temperature: 0.5

Generated Text 1:
the quick brown foxa touss aratrrnoa iup rns rrhrrins mitss iii ina  iss itg bhes etniiia ins twwis ein awtaas ts ens iis iws iis iint tooss rraar rrr rrrrt  aais ttaln 



Generating text with seed: "neural networks"
Temperature: 1.0

Generated Text 2:
neural networks r atu ufmnot nohfi atiiss rosaf  rucs vgtcooss  rnn gbisoh.epn
elsf atitrts mrfv inf attwat. ana
netenis ags ipo.eipuaiavrh a wins tsq o.rsrrv nttouo



Generating text with seed: "deep learning"
Temperature: 0.7

Generated Text 3:
deep learning twwnen g itls eeeraan ttmtls nerrwteg eitnhhoowgl geiwss zi.eure ant smonls  wwrtry etaaonn wvnhs rmwnusg ftid..eeraany oaas eavt s onr s frowrns ore


